In [9]:
import pandas as pd
import numpy as np
import os
from glob import glob
from pathlib import Path

# --- CHEMINS ABSOLUS STATIQUES ---
# 🛑 ATTENTION : Ce chemin doit correspondre EXACTEMENT à la racine de votre projet.
PROJECT_ROOT_ABSOLUTE = "/home/onyxia/work/Gestion-portefeuille/" 

try:
    ROOT_DIR = Path(PROJECT_ROOT_ABSOLUTE)
except Exception:
    ROOT_DIR = Path.cwd() 

# Chemins pour l'entrée et la sortie
RAW_DATA_PATH = ROOT_DIR / "data" / "raw"
INTERIM_DATA_PATH = ROOT_DIR / "data" / "interim"
PROCESSED_DATA_PATH = ROOT_DIR / "data" / "processed"
OUTPUT_FILENAME = "cac40_interim_features.csv"
LAG_WINDOW = 20  # Fenêtre pour les calculs glissants

# --- 1. FONCTION DE CHARGEMENT ET FUSION (CORRECTION DU TYPE DE DONNÉES) ---
def load_and_merge_data(raw_path: Path = RAW_DATA_PATH) -> pd.DataFrame:
    """
    Charge tous les fichiers CSV, les fusionne, et corrige les types de données 
    pour éviter les TypeErrors dans les calculs (conversion forcée en float).
    """
    all_files = glob(str(raw_path / "*.csv")) 
    
    if not all_files:
        print(f"❌ Erreur : Aucun fichier CSV trouvé dans {raw_path}")
        return pd.DataFrame()

    list_df = []
    print(f"📥 Chargement et fusion de {len(all_files)} fichiers...")
    for filename in all_files:
        try:
            # Lire la Date comme une colonne nommée
            # (nécessite que le script de téléchargement utilise df.reset_index(names=['Date']))
            df = pd.read_csv(filename, parse_dates=['Date']) 
            list_df.append(df)
        except Exception as e:
            # Si la lecture échoue, on continue
            print(f"❌ Erreur de chargement pour {filename}: {e}")
            continue
    
    if list_df:
        full_df = pd.concat(list_df)
        
        # --- CORRECTION CLE : Conversion de Type Forcée ---
        # Colonnes qui doivent absolument être numériques pour les calculs
        cols_to_convert = ['Adj Close', 'Volume', 'Dividends']
        
        for col in cols_to_convert:
            if col in full_df.columns:
                # 'errors="coerce"' convertit les strings non numériques en NaN
                full_df[col] = pd.to_numeric(full_df[col], errors='coerce')
            elif col == 'Dividends':
                # Si la colonne 'Dividends' n'existe pas, la créer avec des zéros.
                full_df[col] = 0.0
                print(f"⚠️ Colonne '{col}' ajoutée avec des zéros.")
        
        # Le MultiIndex est créé à partir des colonnes 'Ticker' et 'Date'
        # On s'assure que toutes les lignes ont une valeur Ticker et Date avant de set_index
        full_df = full_df.dropna(subset=['Ticker', 'Date']) 
        full_df = full_df.set_index(['Ticker', 'Date']).sort_index()
        
        # On ne garde que les colonnes pertinentes
        return full_df.loc[:, ['Adj Close', 'Volume', 'Dividends']].copy()
    return pd.DataFrame()

# --- 2. FONCTION DE CALCUL DES FEATURES ---
def compute_financial_features(df: pd.DataFrame) -> pd.DataFrame:
    """ Calcule les indicateurs clés (Returns, Volatility, Momentum, Sharpe) pour le clustering. """
    if df.empty: return df

    # Le calcul est appliqué par groupe de Ticker (groupby(level='Ticker'))
    grouped = df.groupby(level='Ticker')

    print("🔧 Calcul des features de risque et de performance...")
    
    df['Returns'] = grouped['Adj Close'].pct_change()
    
    # Volatilité annualisée (Écart-type glissant)
    df['Volatility'] = grouped['Returns'].transform(
        lambda x: x.rolling(window=LAG_WINDOW).std() * np.sqrt(252)
    )

    # Performance glissante (Momentum)
    df['Performance_20D'] = grouped['Adj Close'].pct_change(LAG_WINDOW)

    # Ratio de Sharpe (Approximation)
    daily_vol_rolling = grouped['Returns'].transform(lambda x: x.rolling(window=LAG_WINDOW).std())
    
    # Évite la division par zéro en remplaçant les zéros dans daily_vol_rolling par NaN ou une petite valeur
    daily_vol_rolling = daily_vol_rolling.replace(0, np.nan) 
    
    df['Sharpe_Ratio_20D'] = grouped['Returns'].transform(
        lambda x: x.rolling(window=LAG_WINDOW).mean() / daily_vol_rolling
    )
    
    # Nettoyage et sélection
    df = df.dropna()
    
    # Réinitialisation de l'index pour que Ticker et Date redeviennent des colonnes
    features_df = df.reset_index()
    
    # Gardons seulement les colonnes finales nécessaires
    return features_df.loc[:, ['Date', 'Ticker', 'Adj Close', 'Volume', 'Returns', 'Volatility', 'Performance_20D', 'Sharpe_Ratio_20D', 'Dividends']]


def run_feature_engineering():
    """ Orchestre le chargement, le calcul et la sauvegarde des features. """
    print("🚀 Étape 2 : Chargement et Feature Engineering...")
    
    # 1. Chargement et fusion des données brutes
    full_data = load_and_merge_data(raw_path=RAW_DATA_PATH)
    
    if full_data.empty:
        print("❌ Processus interrompu: Impossible de charger les données brutes.")
        return

    # 2. Calcul des features
    features_df = compute_financial_features(full_data)
    
    if features_df.empty:
        print("❌ Processus interrompu: Aucune donnée restante après nettoyage des NaN.")
        return

    # 3. Sauvegarde dans le dossier 'interim'
    os.makedirs(INTERIM_DATA_PATH, exist_ok=True)
    output_filepath = os.path.join(INTERIM_DATA_PATH, OUTPUT_FILENAME)
    
    features_df.to_csv(output_filepath, index=False)
    print(f"💾 Feature Engineering terminé. Données sauvées vers : {output_filepath}")

if __name__ == "__main__":
    run_feature_engineering()

🚀 Étape 2 : Chargement et Feature Engineering...
📥 Chargement et fusion de 38 fichiers...
⚠️ Colonne 'Dividends' ajoutée avec des zéros.
🔧 Calcul des features de risque et de performance...
💾 Feature Engineering terminé. Données sauvées vers : /home/onyxia/work/Gestion-portefeuille/data/interim/cac40_interim_features.csv
